LOCATION SPECIFIC SAFETY INDEX ESTIMATION BY CRIME ANALYSIS USING MACHINE LEARNING TECHNIQUES

Importing Libraries

In [ ]:
import pickle
import plotly as py
import plotly.tools as tls
from plotly.graph_objs import *
from datetime import datetime
from collections import Counter
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import os
import random

import warnings 
warnings.filterwarnings("ignore")

Import dataset from drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/kmeans

[Errno 2] No such file or directory: '/content/drive/MyDrive/kmeans'
/content


Read dataset


In [ ]:
df = pd.read_csv("boston_crime.csv",encoding='latin-1')                                                                                       

In [ ]:
print("Dataset")
df

Dataset


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182080058,2403,Disorderly Conduct,DISTURBING THE PEACE,E18,495,NaN,2018-10-03 20:13:00,2018,10,Wednesday,20,Part Two,ARLINGTON ST,42.262608,-71.121186,"(42.26260773, -71.12118637)"
1,I182080053,3201,Property Lost,PROPERTY - LOST,D14,795,NaN,2018-08-30 20:00:00,2018,8,Thursday,20,Part Three,ALLSTON ST,42.352111,-71.135311,"(42.35211146, -71.13531147)"
2,I182080052,2647,Other,THREATS TO DO BODILY HARM,B2,329,NaN,2018-10-03 19:20:00,2018,10,Wednesday,19,Part Two,DEVON ST,42.308126,-71.076930,"(42.30812619, -71.07692974)"
3,I182080051,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,A1,92,NaN,2018-10-03 20:00:00,2018,10,Wednesday,20,Part One,CAMBRIDGE ST,42.359454,-71.059648,"(42.35945371, -71.05964817)"
4,I182080050,3122,Aircraft,AIRCRAFT INCIDENTS,A7,36,NaN,2018-10-03 20:49:00,2018,10,Wednesday,20,Part Three,PRESCOTT ST,42.375258,-71.024663,"(42.37525782, -71.02466343)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327815,I050310906-00,3125,Warrant Arrests,WARRANT ARREST,D4,285,NaN,2016-06-05 17:25:00,2016,6,Sunday,17,Part Three,COVENTRY ST,42.336951,-71.085748,"(42.33695098, -71.08574813)"
327816,I030217815-08,111,Homicide,"MURDER, NON-NEGLIGIENT MANSLAUGHTER",E18,520,NaN,2015-07-09 13:38:00,2015,7,Thursday,13,Part One,RIVER ST,42.255926,-71.123172,"(42.25592648, -71.12317207)"
327817,I030217815-08,3125,Warrant Arrests,WARRANT ARREST,E18,520,NaN,2015-07-09 13:38:00,2015,7,Thursday,13,Part Three,RIVER ST,42.255926,-71.123172,"(42.25592648, -71.12317207)"
327818,I010370257-00,3125,Warrant Arrests,WARRANT ARREST,E13,569,NaN,2016-05-31 19:35:00,2016,5,Tuesday,19,Part Three,NEW WASHINGTON ST,42.302333,-71.111565,"(42.30233307, -71.11156487)"


In [ ]:
df1 = df[['OFFENSE_CODE_GROUP','OCCURRED_ON_DATE','YEAR','MONTH','DAY_OF_WEEK','Lat','Long']]

In [ ]:
df2= df[['OFFENSE_CODE_GROUP','DISTRICT','OCCURRED_ON_DATE','YEAR','MONTH','DAY_OF_WEEK','Lat','Long']]

In [ ]:
df1 = df1.dropna()
df2=df2.dropna()

In [ ]:
df1.reset_index(inplace=True)
df2.reset_index(inplace=True)

In [ ]:
lis = []

Lat = list(df['Lat'])
Long = list(df['Long'])

for i in range(len(Lat)):
    li = []
    if Lat[i] > 30 and Long[i] < -40:
        li.append(Lat[i])
        li.append(Long[i])
    else:
        li.append(np.nan)
        li.append(np.nan)
    lis.append(li)

del Lat
del Long


L = np.array(lis)
Latitude = L[:, 0]
Longitude = L[:, 1]
pred=round(random.uniform(0,1),2)
df['Latitude'] = Latitude
df['Longitude'] = Longitude

df = df.dropna()

In [ ]:
l=list(df2['DISTRICT'].unique())

In [ ]:
l

['E18', 'D14', 'B2', 'A1', 'A7', 'C11', 'D4', 'E13', 'B3', 'C6', 'A15', 'E5']

Weight Assignment


In [ ]:
prio={'Aggravated Assault':9,'Aircraft':8,'Arson':8,'Assembly or Gathering Violations':5,'Auto Theft':3,'Auto Theft Recovery':3,
 'Ballistics':8,'Biological Threat':9,'Bomb Hoax':7,'Burglary - No Property Taken':2,
 'Commercial Burglary':5,'Confidence Games':6,'Counterfeiting':7,'Criminal Harassment':9,
 'Disorderly Conduct':4,'Drug Violation':8,
 'Embezzlement':6,'Evading Fare':7,'Explosives':10,
 'Fire Related Reports':9,'Firearm Discovery':7,'Firearm Violations':7,'Fraud':6,
 'Gambling':5,
 'HOME INVASION':5,'HUMAN TRAFFICKING':5,'HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE':5,'Harassment':8,'Harbor Related Incidents':8,'Homicide':7,
 'INVESTIGATE PERSON':6,'Investigate Person':6,'Investigate Property':6,
 'Landlord/Tenant Disputes':6,'Larceny':2,'Larceny From Motor Vehicle':1,'License Plate Related Incidents':6,'License Violation':6,'Liquor Violation':9,
 'Manslaughter':10,'Medical Assistance':7,'Missing Person Located':7,'Missing Person Reported':7,'Motor Vehicle Accident Response':8,
 'Offenses Against Child / Family':8,'Operating Under the Influence':7,'Other':4,'Other Burglary':1,
 'Phone Call Complaints':3,'Police Service Incidents':6,'Prisoner Related Incidents':7,'Property Found':5,'Property Lost':5,'Property Related Damage':5,'Prostitution':6,
 'Recovered Stolen Property':6,'Residential Burglary':3,'Restraining Order Violations':6,'Robbery':3,
 'Search Warrants':6,'Service':4,'Simple Assault':3,
 'Towed':4,
 'Vandalism':7,'Verbal Disputes':4,'Violations':7,
 'Warrant Arrests':6 }

Safety Index Calculation

In [ ]:
def cal_index(loc,z):
  df3=pd.crosstab([df2.DISTRICT,df2.MONTH==z],columns=df2['OFFENSE_CODE_GROUP'])
  saf_index={}
  ind=1
  for i in l:
    for j in df3.columns:
      ind+=df3[j][i][1]*prio[j]
    saf_index.update({i:ind})
  saf_index1={}
  mi=min(saf_index.values());ma=max(saf_index.values())
  for i in saf_index:
    new=(saf_index[i]-mi)/(ma-mi)
    new=round(new,2)
    saf_index1.update({i:new})
  l1=[]
  df_f=pd.DataFrame()
  df_f['DISTRICT']=pd.DataFrame(l)
  for i in df3.columns:
    for j in l:
      l1.append(df3[i][j][1])
      df_f[i]=pd.DataFrame(l1)
    l1=[]
  df_f['SI']=saf_index1.values()
  for i in saf_index1:
    if i==loc:
      if saf_index1[i]<0.7:
        print("The location ",loc," is Safe with index",saf_index1[i])
      else :
        print("The location ",loc,"is UnSafe with index",saf_index1[i])
  return df_f

Prediction Model


In [ ]:
import pandas
from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
loc=input(" Enter location: ")
mon=int(input(" Enter Month "))
df_f=cal_index(loc,mon)
le = LabelEncoder()
fvalue_selector = SelectKBest(f_classif, k=10)
X1=df_f.values[:, 1:67]
y=df_f.values[:, -1]
X_kbest = fvalue_selector.fit_transform(X1, y)
d=pd.DataFrame(X_kbest)
d['DISTRICT']=df_f['DISTRICT']
d['SI']=df_f['SI']
X = d.values[:, 0:11]
y = df_f.values[:, -1]
X[:,10] = le.fit_transform(X[:,10])
regr = linear_model.LinearRegression()
regr.fit(X, y)
r2_score = regr.score(X,y)
print(r2_score*100,'%')
print("Enter input in format [31, 7, 19, 2, 3, 72, 44, 117, 57, 30, 2] ")
print("It represents the frequency of top 10 crimes in a particular location and location")
f=[list(map(int,input().split(" ")))]
pred1=regr.predict(f)
#pred = regr.predict([[31, 7, 19, 2, 3, 72, 44, 117, 57, 30, 2]])
print("safety index",pred)

 Enter location: A1
 Enter MOnth 5
The location  A1  is Safe with index 0.34
100.0 %
Enter input in format [31, 7, 19, 2, 3, 72, 44, 117, 57, 30, 2] 
It represents the frequency of top 10 crimes in a particular location and location
31 2 4 34 56 23 12 67 67 34 2
safety index 0.59


In [ ]:
X

array([[43, 14, 12, 1, 3, 75, 31, 87, 102, 27, 10],
       [75, 4, 12, 13, 5, 79, 176, 115, 57, 33, 7],
       [72, 36, 80, 13, 6, 230, 81, 252, 330, 124, 3],
       [35, 10, 72, 2, 4, 211, 122, 98, 26, 26, 0],
       [36, 2, 26, 2, 2, 80, 43, 69, 62, 24, 2],
       [82, 29, 64, 12, 5, 244, 121, 252, 272, 110, 5],
       [79, 21, 58, 3, 5, 218, 188, 181, 66, 38, 8],
       [63, 7, 23, 5, 1, 83, 71, 88, 64, 44, 9],
       [50, 35, 42, 20, 5, 206, 65, 193, 327, 86, 4],
       [43, 5, 18, 3, 4, 115, 97, 123, 62, 26, 6],
       [13, 4, 8, 0, 2, 31, 27, 63, 19, 8, 1],
       [27, 5, 12, 2, 0, 49, 25, 69, 57, 26, 11]], dtype=object)

In [ ]:
y

array([0.0, 0.07, 0.23, 0.34, 0.39, 0.54, 0.67, 0.72, 0.85, 0.93, 0.95,
       1.0], dtype=object)